<a href="https://colab.research.google.com/github/comojin1994/DenseNet/blob/master/Exercise_DenseNet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%tensorflow_version 2.x
import tensorflow as tf
import numpy as np
tf.__version__

TensorFlow 2.x selected.


'2.1.0'

### Hyperparameter

In [0]:
EPOCHS = 10
batch_size = 32

### Dense Unit

In [0]:
class DenseUnit(tf.keras.Model):
    def __init__(self, filter_out, kernel_size):
        super(DenseUnit, self).__init__()
        self.bn1 = tf.keras.layers.BatchNormalization()
        self.conv1 = tf.keras.layers.Conv2D(filter_out, kernel_size, padding='same')

        self.bn2 = tf.keras.layers.BatchNormalization()
        self.conv2 = tf.keras.layers.Conv2D(filter_out, kernel_size, padding='same')

        self.concat = tf.keras.layers.Concatenate()

    def call(self, x, training=False, mask=None):
        h = self.bn1(x, training=training)
        h = tf.nn.relu(h)
        h = self.conv1(h)
        h = self.bn2(h, training=training)
        h = tf.nn.relu(h)
        h = self.conv2(h)
        h = self.concat([x, h])
        return h

### Dense Layer

In [0]:
class DenseLayer(tf.keras.Model):
    def __init__(self, num_units, growth_rate, kernel_size):
        super(DenseLayer, self).__init__()
        self.sequence = list()
        for _ in range(num_units):
            self.sequence.append(DenseUnit(growth_rate, kernel_size))

    def call(self, x, training=False, mask=None):
        for unit in self.sequence:
            x = unit(x, training=training)
        return x

### Transition Layer

In [0]:
class TransitionLayer(tf.keras.Model):
    def __init__(self, filters, kernel_size):
        super(TransitionLayer, self).__init__()
        self.conv = tf.keras.layers.Conv2D(filters, kernel_size, padding='same')
        self.pool = tf.keras.layers.MaxPool2D()
 
    def call(self, x):
        x = self.conv(x)
        x = self.pool(x)
        return x

### Model

In [0]:
class DenseNet(tf.keras.Model):
    def __init__(self):
        super(DenseNet, self).__init__()
        self.conv1 = tf.keras.layers.Conv2D(8, (3, 3), padding='same', activation='relu') # 32x32x8

        self.dl1 = DenseLayer(2, 4, (3, 3)) # 32x32x16
        self.t1 = TransitionLayer(16, (3, 3)) # 16x16x16

        self.dl2 = DenseLayer(2, 8, (3, 3)) # 16x16x32
        self.t2 = TransitionLayer(32, (3, 3)) # 8x8x32

        self.dl3 = DenseLayer(2, 16, (3, 3)) # 8x8x64
        self.t3 = TransitionLayer(64, (3, 3)) # 4x4x64

        self.dl4 = DenseLayer(2, 32, (3, 3)) # 4x4x128
        self.flatten = tf.keras.layers.Flatten() # 2048
        self.dense1 = tf.keras.layers.Dense(1024, activation='relu')
        self.dense2 = tf.keras.layers.Dense(10, activation='softmax')

    def call(self, x, training=False, mask=None):
        x = self.conv1(x)
        x = self.dl1(x, training=training)
        x = self.t1(x)
        x = self.dl2(x, training=training)
        x = self.t2(x)
        x = self.dl3(x, training=training)
        x = self.t3(x)
        x = self.dl4(x, training=training)
        x = self.flatten(x)
        x = self.dense1(x)
        x = self.dense2(x)
        return x

### Load Data

In [9]:
cifar10 = tf.keras.datasets.cifar10
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
x_train, x_test = x_train / 255., x_test / 255.
x_train = x_train.astype(np.float32)
x_test = x_test.astype(np.float32)

train_ds = tf.data.Dataset.from_tensor_slices((x_train, y_train)).shuffle(len(x_train)).batch(batch_size)
test_ds = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(batch_size)

170500096/170498071 [==============================] - 2s 0us/step


### Define train, test loop

In [0]:
@tf.function
def train_step(model, images, labels, loss_object, optimizer, train_loss, train_accuracy):
    with tf.GradientTape() as tape:
        predictions = model(images, training=True)
        loss = loss_object(labels, predictions)
    gradients = tape.gradient(loss, model.trainable_variables)

    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    train_loss(loss)
    train_accuracy(labels, predictions)

@tf.function
def test_step(model, images, labels, loss_object, test_loss, test_accuracy):
    predictions = model(images, training=False)
    t_loss = loss_object(labels, predictions)

    test_loss(t_loss)
    test_accuracy(labels, predictions)

### Define model, loss function, optimizer and metrcs

In [0]:
model = DenseNet()

loss_object = tf.keras.losses.SparseCategoricalCrossentropy()
optimizer = tf.keras.optimizers.Adam()

train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')
test_loss = tf.keras.metrics.Mean(name='test_loss')
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='test_accuracy')

### Training

In [17]:
for epoch in range(EPOCHS):
    for images, labels in train_ds:
        train_step(model, images, labels, loss_object, optimizer, train_loss, train_accuracy)

    for images, labels in test_ds:
        test_step(model, images, labels, loss_object, test_loss, test_accuracy)

    template = 'Epoch {}, Loss: {}, Accuracy: {}, Test Loss: {}, Test Accuracy: {}'
    print(template.format(
          epoch + 1,
          train_loss.result(),
          train_accuracy.result() * 100,
          test_loss.result(),
          test_accuracy.result() * 100
    ))

Epoch 1, Loss: 1.3312525749206543, Accuracy: 52.53999710083008, Test Loss: 1.0738935470581055, Test Accuracy: 62.910003662109375
Epoch 2, Loss: 1.1014732122421265, Accuracy: 61.01899719238281, Test Loss: 0.9922609925270081, Test Accuracy: 66.04000091552734
Epoch 3, Loss: 0.9691203236579895, Accuracy: 65.84400177001953, Test Loss: 0.934353232383728, Test Accuracy: 67.99333190917969
Epoch 4, Loss: 0.870938241481781, Accuracy: 69.36000061035156, Test Loss: 0.8978102803230286, Test Accuracy: 69.37750244140625
Epoch 5, Loss: 0.7932301759719849, Accuracy: 72.1144027709961, Test Loss: 0.9087613821029663, Test Accuracy: 69.89199829101562
Epoch 6, Loss: 0.7263341546058655, Accuracy: 74.49766540527344, Test Loss: 0.8958187103271484, Test Accuracy: 70.7066650390625
Epoch 7, Loss: 0.6665540933609009, Accuracy: 76.61885833740234, Test Loss: 0.9098419547080994, Test Accuracy: 71.17428588867188
Epoch 8, Loss: 0.6162998676300049, Accuracy: 78.40999603271484, Test Loss: 0.9204843640327454, Test Accurac

In [18]:
model.summary()

Model: "dense_net"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              multiple                  224       
_________________________________________________________________
dense_layer (DenseLayer)     multiple                  1136      
_________________________________________________________________
transition_layer (Transition multiple                  2320      
_________________________________________________________________
dense_layer_1 (DenseLayer)   multiple                  4288      
_________________________________________________________________
transition_layer_1 (Transiti multiple                  9248      
_________________________________________________________________
dense_layer_2 (DenseLayer)   multiple                  16640     
_________________________________________________________________
transition_layer_2 (Transiti multiple                  36

In [20]:
model.compile(optimizer=optimizer, loss=loss_object, metrics=['accuracy'])
model.evaluate(test_ds)

313/313 [==============================] - 3s 11ms/step - loss: 1.2651 - accuracy: 0.7317


[1.2650999904821476, 0.7317]